In [1]:
import webbrowser
import time
import os
import shutil
import copy
import re
import csv
import pandas as pd

In [2]:
keyword = "Iguala+violiencia+muertos"

path = '/Users/martin/Downloads'

scrapings_dir = 'gt_{0}'.format(keyword)
if not os.path.exists(path+"/"+scrapings_dir):
    os.makedirs(path+"/"+scrapings_dir)

In [3]:
def ScrapeTwoMonths(keyword, year, startmonth):
    print 'Scraping month'+str(startmonth)+' in '+str(year)
    URL_start = "http://www.google.com/trends/trendsReport?&q="
    URL_end = "&cmpt=q&content=1&export=1"
    queries = keyword[0]
    if len(keyword) > 1:
        queries_list = []
        for i in range(0,len(keyword)):
            queries_list.append(keyword[i])
        queries = '%20'.join(queries_list)  
    date = '&date='+str(startmonth)+'%2F'+str(year)+'%202m'
    URL = URL_start+queries+date+URL_end
    webbrowser.open(URL)    


def ScrapeRange(keyword, startmonth, startyear, endmonth, endyear):   
    for i in range(startmonth,11,2):
        ScrapeTwoMonths(keyword,startyear,i)
        time.sleep(7)
    for y in range(startyear + 1, endyear):
        for i in range(1,11,2):
            ScrapeTwoMonths(keyword,y,i)
            time.sleep(7)
    for i in range(1,endmonth,2):
        ScrapeTwoMonths(keyword,endyear,i)
        time.sleep(7)
    files = copy.deepcopy(os.listdir(path))    
    for i in range(0,len(files)):
        if files[i].lower().endswith('.csv'):
            try:
                if files[i][-5] == ")":
                    oldname = path+'/'+files[i]
                    newname = path+'/report'+files[i][-6]+'.csv'
                    os.rename(oldname,newname)
            except OSError:
                pass

    quarterly_files = [fn for fn in os.listdir(path) if fn.lower().startswith('report')]                  
    for file in quarterly_files:
        shutil.move(path+"/"+file,path+'/'+scrapings_dir)
    full_path = path+'/'+scrapings_dir    
    newfiles = copy.deepcopy(os.listdir(full_path))
    for i in range(0,len(newfiles)):
        oldname = full_path+'/'+newfiles[i]
        if os.path.getsize(oldname) < 800:
            print 'File '+oldname+' is unusually small...'
        newname = full_path+'/'+str(os.path.getmtime(full_path+'/'+newfiles[i]))[:-2]+".csv"
        os.rename(oldname, newname) 
    
    
def CreateDailyFrame():
    files = copy.deepcopy(os.listdir(path+'/'+scrapings_dir))[:-1]
    print files
    date_pattern = re.compile('\d\d\d\d-\d\d-\d\d')
    for i in range(0,len(files)):
        if files[i].lower().endswith('.csv'):
            oldname = path+'/'+scrapings_dir+'/'+files[i]
            newname = path+'/'+scrapings_dir+'/'+'bimonthly'+str(i)+'.csv'
            temp_file = csv.reader(open(oldname,'ru'))
            with open(newname,'wb') as write_to:
                write_data = csv.writer(write_to, delimiter=',')
                for row in temp_file:
                    if len(row)==2:
                        if re.search(date_pattern,row[0]) is not None:
                            write_data.writerows([row])
            os.remove(oldname)
    files = [fn for fn in copy.deepcopy(os.listdir(path+'/'+scrapings_dir))[:-1] if fn.lower().startswith('bimonthly')]
    frames_list = []
    for file in files:
        df = pd.read_csv(path+'/'+scrapings_dir+'/'+file,index_col=None,header=None)
        list.append(df)
    frame = pd.concat(list,ignore_index=True)
    return frame    
    
    
def CreateWeeklyFrame():
        date_pattern = re.compile('\d\d\d\d-\d\d-\d\d\s-\s\d\d\d\d-\d\d-\d\d')
        oldname = path+'/'+scrapings_dir+'/'+'weekly_data.csv'
        newname = path+'/'+scrapings_dir+'/'+'weekly.csv'
        temp_file = csv.reader(open(oldname,'ru'))
        with open(newname,'wb') as write_to:
            write_data = csv.writer(write_to, delimiter=',')
            for row in temp_file:
                if len(row) == 2:
                    if re.search(date_pattern,row[0]) is not None:
                        write_data.writerows([row])
        os.remove(oldname)
        frame = pd.read_csv(newname,index_col=None,header=None)
        return frame
    
def StitchFrames():
    daily_frame = CreateDailyFrame()
    interim_weekly_frame = CreateWeeklyFrame()
    daily_frame.columns = ['Date', 'Daily_Volume']
    pd.to_datetime(daily_frame['Date'])
    interim_weekly_frame.columns = ['Date_Range', 'Weekly_Volume']
    date_pattern = re.compile('\d\d\d\d-\d\d-\d\d')
    startdates = []
    enddates = []
    for i in range(0,len(interim_weekly_frame['Date_Range'])):
        startdates.append(re.findall(date_pattern,interim_weekly_frame['Date_Range'][i])[0])
        enddates.append(re.findall(date_pattern,interim_weekly_frame['Date_Range'][i])[1])
    weekly_frame = pd.DataFrame(data=[startdates,enddates,interim_weekly_frame['Weekly_Volume'].tolist()]).transpose()
    weekly_frame.columns = ['Start_Date', 'End_Date', 'Weekly_Volume']
    pd.to_datetime(weekly_frame['Start_Date'])
    pd.to_datetime(weekly_frame['End_Date'])   
    

In [5]:
bins = []

for i in range(0,len(weekly_frame)):
    bins.append(pd.date_range(weekly_frame['Start_Date'][i],periods=7,freq='d'))

NameError: name 'weekly_frame' is not defined

In [4]:
weekly_frame = weekly_frame.set_index('Start_Date')
daily_frame = daily_frame.set_index('Date')
final_data = {}

for i in range(0,len(bins)):
    for j in range(0,len(bins[i])):
        final_data[bins[i][j]] = weekly_frame['Weekly_Volume'][str(bins[i][0].date())]*daily_frame['Daily_Volume'][str(bins[i][j].date())]/daily_frame['Daily_Volume'][str(bins[i][0].date())]
        
final_data_frame = DataFrame.from_dict(final_data,orient='index').sort()
    final_data_frame[0] = np.round(final_data_frame[0]/final_data_frame[0].max()*100)

    final_data_frame.columns=['Volume']
    final_data_frame.index.names = ['Date']

    final_name = path+'/'+scrapings_dir+'/'+'final_output.csv'

    final_data_frame.to_csv(final_name, sep=',')

NameError: name 'weekly_frame' is not defined

In [24]:
def get_links(terms,category):
    category_articles = []
    article_count = int(category_total/len(terms))
    for term in terms:
        category_articles.extend(ex.google_news_search(term,category,article_count))
    return category_articles

# Policy Articles
#violence_terms = ['Iguala','Deaths','killed','violence','crime']
violence_terms = ['San Francisco','Deaths','killed','violence','crime']

violence = get_links(violence_terms,'Violence')
print len(violence)
article_links.extend(violence)


# Extract Content & Create Corpus
print "Total %d links to extract" % len(article_links)

150
Total 420 links to extract


In [25]:
ex.generate_corpus(article_links)
print ex.get_stats()

Extracting  content from links...
Saving article Mom's lawyer: Murder charges in babies' deaths 'speculation'...
Record already inserted with title Mom's lawyer: Murder charges in babies' deaths 'speculation' 
Saving article Mass. opioid overdose deaths up 228 percent, new state report says...
Saving article Everything you know about celebrity deaths is wrong...
Record already inserted with title Everything you know about celebrity deaths is wrong 
Saving article Man Charged With Murder in Shooting Deaths of Ex-Girlfriend, Her New Boyfriend in Winnetka...
Record already inserted with title Man Charged With Murder in Shooting Deaths of Ex-Girlfriend, Her New Boyfriend in Winnetka 
Saving article Eating healthy fats may reduce death from heart disease...
Record already inserted with title Eating healthy fats may reduce death from heart disease 
Saving article Diane Staudte pleads guilty for antifreeze deaths of husband, son...
Record already inserted with title Diane Staudte pleads guilt

OperationalError: no such table: articles

In [4]:
# Open/Create a file to append data
csvFile = open('tweets.csv', 'a')

In [3]:
# Make the graphs prettier
pd.set_option('display.mpl_style', 'default')

In [4]:
WORDS = ['robbery', 'assault', 'brivery', 'thief']

In [5]:
cKey='7eDCo2qRKjSc9QTlNNo6pqc37' 
cSecret = '14qubum20DqXkVYMFsqVJC188itHF10QEmI1YZQDVbAtxq3Uw0'

aKey='3179962074-VTtUFiexeL9uG7LcTqiFfRDkf9n7nq0bmd5Nx4y' 
aSecret = '3179962074-VTtUFiexeL9uG7LcTqiFfRDkf9n7nq0bmd5Nx4y'





In [6]:
#Use tweepy.OAuthHandler to create an authentication using the given key and secret
auth = tweepy.OAuthHandler(consumer_key=cKey, consumer_secret=cSecret)

#Connect to the Twitter API using the authentication
api = tweepy.API(auth)

In [8]:
# Open/Create a file to append data
csvFile = open('tweets.csv', 'a')

#Use csv Writer
csvWriter = csv.writer(csvFile)
count = 0

for tweet in tweepy.Cursor(api.search,q="%23robbery",
                           lang="en").items(10):
    #if '%23robbery' in item:  ##,since="2014-01-01", until="2014-12-01"
    #    count += 1             ## %20since%3A2014-09-09%20until%3A2014-10-25
    #elif 'message' in item and item['code'] == 88:
    #    print('SUSPEND, RATE LIMIT EXCEEDED: %s' % item['message'])
    #    break
    print tweet.created_at, tweet.text, tweet.user.location
    
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8'),tweet.user.location])
    
#print count

2016-01-18 23:30:22 Just witnessed stones drawing 1-1 with gosport, i could have been watching England v Poland 1973!#robbery Enfield, north london
2016-01-18 23:19:01 @TJDillashaw - watched the fight and I don't get it either. Gotta take it from champ n it didn't happen. #UFCBoston #robbery 
2016-01-18 22:51:45 @Bell chargin me 4 a month of UNUSED service. Incompetent employees fail to properly assist customers. #robbery #crook #bellcanada #notcool Scarborough
2016-01-18 22:49:58 RT : MarekRamadan: RT ScoutForTrout: Classic grandmothers! #Robbery #Nan #Food https://t.co/MAMatDiSSC 
2016-01-18 22:47:46 RT @ScoutForTrout: Classic grandmothers! #Robbery #Nan #Food https://t.co/LxU2Wh1Ndq The Hague, South Holland
2016-01-18 21:28:12 @ThisQuestions4 @TJDillashaw @TheDomin8r @GrabakaHitman Cruz won't stand in the pocket bang anyway so... #UFCFightNight81  #robbery TEXAS 
2016-01-18 21:04:40 Armed #robbery #suspect found hiding in girlfriend's closet - Jan 18 @ 4:04 PM ET  https://t.co/yOR3k

In [99]:
#Perform a basic search query where we search for the '#theft' in the tweets
#result = api.search(q='%23robbery') #%23 is used to specify '#'


# Print the number of items returned by the search query to verify our query ran. 
# Its 15 by default
#len(result)

In [69]:
tweet = result[0] #Get the first tweet in the result


# Analyze the data in one tweet to see what we require
#for param in dir(tweet):
#The key names beginning with an '_' are hidden ones and usually not required, 
# so we'll skip them
    #if not param.startswith("_"):
        #print "%s : %s\n" % (param, eval('tweet.'+param))

In [ ]:
results = []

#Get the first 5000 items based on the search query
for tweet in tweepy.Cursor(api.search, q='%theft'&since:2011-05-16&until:2011-08-16).items(5000):
    results.append(tweet)

# Verify the number of items returned
print len(results)       

In [35]:
count = 0
r = TwitterRestPager(api, 'search/tweets', {'q':'%23robbery', 'count':100})
for item in r.get_iterator(wait=6):
    if 'text' in item:
        count += 1
    elif 'message' in item and item['code'] == 88:
        print('SUSPEND, RATE LIMIT EXCEEDED: %s' % item['message'])
        break
print(count)

AttributeError: 'API' object has no attribute 'request'

In [27]:
# Create a function to convert a given list of tweets into a Pandas DataFrame.
# The DataFrame will consist of only the values, which I think might be useful for 
# analysis...

def toDataFrame(tweets):

    DataSet = pd.DataFrame()

    DataSet['tweetID'] = [tweet.id for tweet in tweets]
    DataSet['tweetText'] = [tweet.text for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['tweetCreated'] = [tweet.created_at for tweet in tweets]
    DataSet['userID'] = [tweet.user.id for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userName'] = [tweet.user.name for tweet in tweets]
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]

    return DataSet

#Pass the tweets list to the above function to create a DataFrame
DataSet = toDataFrame(results)

In [28]:
# Let's check the top 5 records in the Data Set
DataSet.head(5)    


,tweetID,tweetText,tweetRetweetCt,tweetFavoriteCt,tweetSource,tweetCreated,userID,userScreen,userName,userCreateDt,userDesc,userFollowerCt,userFriendsCt,userLocation,userTimezone
0,687780326836428800,RT @FarmCrimeUK: @POSITIVEFARMERS could you po...,1,0,Twitter for Android,2016-01-14 23:36:22,3940311377,RAWfarms,OADmilking Rich,2015-10-12 20:46:53,#OADmilking 400 happy cows. Farming #soil for ...,460,905,Pembrokeshire,None
1,687780326685454344,So. @NickBolesMP is now justifying the Tory th...,0,0,Twitter Web Client,2016-01-14 23:36:22,353022633,EyeEdinburgh,Jane Carnall,2011-08-11 12:46:30,I blog as EdinburghEye. Here & on my blog I re...,3031,2601,Edinburgh,Edinburgh
2,687780255206105088,PRIOR 2 murder CHILD ENDANGERMENT ID theft MA...,0,0,SocialOomph,2016-01-14 23:36:05,4061763478,bullybully1323,bully,2015-10-28 16:14:21,,51,39,,None
3,687780253444521984,PRIOR 2 murder CHILD ENDANGERMENT ID theft MA...,0,0,SocialOomph,2016-01-14 23:36:05,4061505257,justice4all26,justice4all,2015-10-28 15:35:39,,86,49,,None
4,687780241666916352,RT @glebuscesya: Las Vegas police are investig...,3,0,App48585185,2016-01-14 23:36:02,3209775688,nanonnatale,Milagros Raab,2015-04-26 22:04:12,,87,87,,None


In [29]:
# Let's check the top 5 records in the Data Set
DataSet.head(5)

,tweetID,tweetText,tweetRetweetCt,tweetFavoriteCt,tweetSource,tweetCreated,userID,userScreen,userName,userCreateDt,userDesc,userFollowerCt,userFriendsCt,userLocation,userTimezone
0,687780326836428800,RT @FarmCrimeUK: @POSITIVEFARMERS could you po...,1,0,Twitter for Android,2016-01-14 23:36:22,3940311377,RAWfarms,OADmilking Rich,2015-10-12 20:46:53,#OADmilking 400 happy cows. Farming #soil for ...,460,905,Pembrokeshire,None
1,687780326685454344,So. @NickBolesMP is now justifying the Tory th...,0,0,Twitter Web Client,2016-01-14 23:36:22,353022633,EyeEdinburgh,Jane Carnall,2011-08-11 12:46:30,I blog as EdinburghEye. Here & on my blog I re...,3031,2601,Edinburgh,Edinburgh
2,687780255206105088,PRIOR 2 murder CHILD ENDANGERMENT ID theft MA...,0,0,SocialOomph,2016-01-14 23:36:05,4061763478,bullybully1323,bully,2015-10-28 16:14:21,,51,39,,None
3,687780253444521984,PRIOR 2 murder CHILD ENDANGERMENT ID theft MA...,0,0,SocialOomph,2016-01-14 23:36:05,4061505257,justice4all26,justice4all,2015-10-28 15:35:39,,86,49,,None
4,687780241666916352,RT @glebuscesya: Las Vegas police are investig...,3,0,App48585185,2016-01-14 23:36:02,3209775688,nanonnatale,Milagros Raab,2015-04-26 22:04:12,,87,87,,None


In [30]:
# 'None' is treated as null here, so I'll remove all the records having 'None' in their 'userTimezone' column
DataSet = DataSet[DataSet.userTimezone.notnull()]

# Let's also check how many records are we left with now
len(DataSet)


652

In [31]:
# Count the number of tweets in each time zone and get the first 10
tzs = DataSet['userTimezone'].value_counts()[:10]
print tzs

Pacific Time (US & Canada)     130
Eastern Time (US & Canada)     113
Central Time (US & Canada)      58
London                          52
Amsterdam                       49
West Central Africa             26
Hawaii                          18
Mountain Time (US & Canada)     15
Arizona                         14
Quito                           14
Name: userTimezone, dtype: int64


In [32]:
# Create a bar-graph figure of the specified size
plt.rcParams['figure.figsize'] = (15, 5)

# Plot the Time Zone data as a bar-graph
tzs.plot(kind='bar')


# Assign labels and title to the graph to make it more presentable
plt.xlabel('Timezones')
plt.ylabel('Tweet Count')
plt.title('Top 10 Timezones tweeting about #Oscars2015')


In [42]:
import vincent
world_countries = r'world-countries.json'
world = vincent.Map(width=120, height=1000)
world.geo_data(projection='winkel3', scale=200, world=world_countries)
world.to_json('/home/martin/Dropbox/a_Insight/Project/world.jpg')

vis = vincent.Map(width=1000, height=800)
#Add the US county data and a new line color
vis.geo_data(projection='albersUsa', scale=1000, counties=county_geo)
vis + ('2B4ECF', 'marks', 0, 'properties', 'enter', 'stroke', 'value')

#Add the state data, remove the fill, write Vega spec output to JSON
vis.geo_data(states=state_geo)
vis - ('fill', 'marks', 1, 'properties', 'enter')
vis.to_json(path)

TypeError: 'NoneType' object is not iterable